# Otimizando Regressor LGBM para Predição das Notas de Redação

In [13]:
## --- Bibliotecas para estrutura de dados
import pandas as pd
import numpy as np

## --- Funções de pre-processamento
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

## --- Bibliotecas de machine learning
from sklearn.cluster import KMeans
import lightgbm as lgbm

## --- Funções definidas pelo usuário
from subroutines import reduce_mem_usage

import warnings
import pickle
warnings.filterwarnings('ignore')

In [14]:
## variáveis relevantes para leitura

## --- notas

notas = ['NU_NOTA_REDACAO']

### --- variáveis gerais
list_vars = np.array(['Q001','Q002', 'Q003', 'Q004', 'Q005', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011','Q012', 'Q013',
            'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021','Q022', 'Q023', 'Q024', 'Q025',
            'IN_ACESSO', 'TP_ANO_CONCLUIU','TP_SEXO', 'TP_DEPENDENCIA_ADM_ESC','TP_LINGUA',
            'NU_IDADE', 'TP_ESCOLA', 'TP_COR_RACA', 'TP_ST_CONCLUSAO', 'IN_LIBRAS',
            'CO_MUNICIPIO_RESIDENCIA', 'CO_ESCOLA', 'CO_MUNICIPIO_PROVA',
            'TP_ENSINO', 'SG_UF_PROVA', 'TP_ESTADO_CIVIL', 'TP_NACIONALIDADE',
            'IN_SEM_RECURSO', 'IN_SALA_ESPECIAL', 'SG_UF_NASCIMENTO', 'SG_UF_ESC',
            'IN_TREINEIRO', 'IN_DEFICIT_ATENCAO', 'TP_SIT_FUNC_ESC',
            'CO_MUNICIPIO_ESC', 'IN_LEDOR', 'IN_TEMPO_ADICIONAL',
            'IN_DEFICIENCIA_AUDITIVA', 'TP_LOCALIZACAO_ESC', 'IN_DEFICIENCIA_MENTAL',
            'IN_SURDEZ', 'IN_AUTISMO', 'IN_DEFICIENCIA_FISICA', 'IN_TRANSCRICAO',
            'CO_MUNICIPIO_NASCIMENTO', 'CO_UF_NASCIMENTO', 'CO_UF_PROVA',
            'IN_MAQUINA_BRAILE', 'TP_PRESENCA_MT', 'TP_PRESENCA_LC',
            'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_STATUS_REDACAO'])

In [27]:
## --- lendo dados de treino
reader = pd.read_csv('../input/train.csv', engine='c', chunksize=50000, 
                     nrows=350000, usecols=np.append(list_vars, notas) )


df = pd.DataFrame(columns=pd.read_csv('../input/train.csv', nrows=2, 
                                      usecols=np.append(list_vars, notas)).columns)


for chunk in reader:
    df = pd.concat([df ,reduce_mem_usage(chunk)])

In [28]:
df['NU_IDADE'] = df['NU_IDADE'].fillna(df['NU_IDADE'].mode().iloc[0])
df['CO_MUNICIPIO_NASCIMENTO'] = df['CO_MUNICIPIO_NASCIMENTO'].fillna(df['CO_MUNICIPIO_PROVA'])
df['CO_MUNICIPIO_ESC'] = df['CO_MUNICIPIO_ESC'].fillna(df['CO_MUNICIPIO_PROVA'])
df['TP_SIT_FUNC_ESC'] = df['TP_SIT_FUNC_ESC'].fillna(1)
df['TP_LOCALIZACAO_ESC'] = df['TP_LOCALIZACAO_ESC'].fillna(1)
df['SG_UF_ESC'] = df['SG_UF_ESC'].fillna(df['SG_UF_NASCIMENTO'])

In [29]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
pd.set_option('display.max_rows', df.shape[0]+1)
missing_value_df

,column_name,percent_missing
CO_MUNICIPIO_RESIDENCIA,CO_MUNICIPIO_RESIDENCIA,0.000000
NU_IDADE,NU_IDADE,0.000000
TP_SEXO,TP_SEXO,0.000000
TP_ESTADO_CIVIL,TP_ESTADO_CIVIL,0.000000
TP_COR_RACA,TP_COR_RACA,0.000000
TP_NACIONALIDADE,TP_NACIONALIDADE,0.000000
CO_MUNICIPIO_NASCIMENTO,CO_MUNICIPIO_NASCIMENTO,0.000000
CO_UF_NASCIMENTO,CO_UF_NASCIMENTO,2.870857
SG_UF_NASCIMENTO,SG_UF_NASCIMENTO,2.870857
TP_ST_CONCLUSAO,TP_ST_CONCLUSAO,0.000000


### Feature Engineering

In [30]:

## --- variáveis para codificar
list_toenc = ['Q001','Q002', 'Q003', 'Q004', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011','Q012', 'Q013',
            'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021','Q022', 'Q023', 'Q024', 'Q025',
            'CO_MUNICIPIO_RESIDENCIA', 'CO_ESCOLA', 'CO_MUNICIPIO_PROVA',
            'SG_UF_PROVA', 'SG_UF_NASCIMENTO', 'SG_UF_ESC','TP_LINGUA','TP_SEXO',
            'CO_MUNICIPIO_ESC', 'CO_MUNICIPIO_NASCIMENTO', 'CO_UF_NASCIMENTO', 'CO_UF_PROVA']

## --- variáveis mais relevantes para redacao
#ft_rd = np.array(['TP_STATUS_REDACAO', 'Q006', 'NU_IDADE', 'Q024', 'Q008', 'TP_ESCOLA', 'Q004', 'TP_ST_CONCLUSAO', 'Q002', 'TP_ANO_CONCLUIU', 'TP_DEPENDENCIA_ADM_ESC', 'Q003', 'Q001', 'Q022', 'Q010','TP_LINGUA', 'Q007', 'Q025', 'Q009', 'Q019', 'TP_ESTADO_CIVIL', 'IN_TREINEIRO', 'Q016', 'Q018', 'CO_ESCOLA', 'TP_LOCALIZACAO_ESC', 'TP_ENSINO', 'CO_MUNICIPIO_ESC', 'Q013', 'CO_MUNICIPIO_PROVA', 'CO_MUNICIPIO_RESIDENCIA', 'CO_UF_PROVA', 'Q021', 'SG_UF_ESC', 'Q014','TP_SEXO', 'TP_SIT_FUNC_ESC', 'CO_UF_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO', 'TP_COR_RACA', 'Q023', 'SG_UF_NASCIMENTO', 'SG_UF_PROVA', 'Q020', 'Q012', 'IN_TEMPO_ADICIONAL', 'IN_DEFICIT_ATENCAO', 'Q005', 'Q011', 'TP_NACIONALIDADE','Q015', 'Q017'])
ft_rd = list_vars


ft_clust = ['Q006', 'NU_IDADE', 'Q024', 'TP_ESCOLA', 'Q004', 'Q002', 'Q003']

In [31]:
## --- encoder
is_to_enc = 0
if is_to_enc == 1:
    ##encoding
    enc1 = reduce_mem_usage( pd.read_csv('../input/train.csv', engine='c',
                                       usecols=list_toenc) )
    enc2 = reduce_mem_usage( pd.read_csv('../input/test.csv', engine='c',
                                       usecols=list_toenc) )

    enc = pd.concat([enc1,enc2])
    del enc1, enc2
    encoders = []
    for coluna in list_toenc:
        if enc[coluna].isna().any() == True:
            encoders.append( LabelEncoder().fit( list(set(enc[coluna].astype(str).fillna("missing").replace("nan", "missing").unique().tolist())) ) )
        else:
            encoders.append( LabelEncoder().fit( list(set(enc[coluna].astype(str).unique().tolist())) ) )
    del enc
    ## saving encoders
    for enc, n in zip( encoders, np.arange(len(encoders)) ):
        np.save('./Encoders/redacao/classes_%s.npy' % n, enc.classes_)
else:
    encoders = []
    for n in np.arange(len(list_toenc)):
        enc = LabelEncoder()
        enc.classes_ = np.load('./Encoders/redacao/classes_%s.npy' % n)
        encoders.append(enc)

In [32]:
## --- substitui notas missing por zero
for coluna in notas:
    df[coluna] = df[coluna].fillna(0)
    
## --- substitui valores NaN (variáveis numéricas)  por inteiro arbitrário 

for coluna in df[list_vars].loc[:2,~df[list_vars].columns.isin(list_toenc)].columns:
    df[coluna] = df[coluna].fillna(-32768).astype('int16')
    
i=0
for coluna in list_toenc:
    df[coluna] = df[coluna].astype(str).fillna("missing").replace("nan", "missing").astype('category')
    df[coluna] = encoders[i].transform(df[coluna])
    i+=1

In [33]:
## --- redução de cardinalidade
df['Q004'] = df['Q004'].apply(lambda x: 0 if x==0 else
                                        1 if (x==1) | (x==2) | (x==5) else x-1)
df['Q024'] = df['Q024'].apply(lambda x: x if x<3 else 3)

### Treino do Modelo

In [34]:
## --- instanciamento do agrupador
nclust = 3
model = KMeans(n_clusters=nclust)
model.fit(df[ft_clust])

# atribui cada amostra a um grupo
df['CLUSTER'] = model.predict(df[ft_clust])

In [35]:
## --- regressor lbmg
regressor_red = make_pipeline(StandardScaler(), lgbm.LGBMRegressor(boosting_type='gbdt', 
                                                                   learning_rate=0.07, 
                                                                   max_depth=-1, 
                                                                   n_estimators=350))

In [36]:
kf = KFold(n_splits=5, random_state=3, shuffle=True)

xval_err = 0
y = df['NU_NOTA_REDACAO']
for train_index, test_index in kf.split(df[np.append(ft_rd,'CLUSTER')]):
    X_train, X_test = df[np.append(ft_rd,'CLUSTER')].iloc[train_index,:], df[np.append(ft_rd,'CLUSTER')].iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    
    regressor_red.fit(X_train, y_train)
    y_pred = pd.Series(regressor_red.predict(X_test))
    #y_pred.iloc[X_test[X_test['TP_PRESENCA_CH']!=1].reset_index(drop=True).index] = 0
    xval_err += np.sqrt(mean_squared_error(y_test, y_pred))

rmse_5cv = xval_err/5
rmse_5cv

114.02519456806563

In [14]:
"""## --- fit do modelo e gravação dos parâmetros em arquivo
regressor_red.fit(df[np.append(ft_rd,'CLUSTER')],
                  df['NU_NOTA_REDACAO'].astype('int16'))

pickle.dump(model, open('./models/redacao.sav', 'wb'))

#del df"""

"## --- fit do modelo e gravação dos parâmetros em arquivo\nregressor_red.fit(df[np.append(ft_rd,'CLUSTER')],\n                  df['NU_NOTA_REDACAO'].astype('int16'))\n\npickle.dump(model, open('./models/redacao.sav', 'wb'))\n\n#del df"

#### Avaliação de Desempenho

In [15]:
X_train, X_test, y_train, y_test = train_test_split(df[np.append(ft_rd,'CLUSTER')],
                                                    df['NU_NOTA_REDACAO'].astype('int16'),
                                                    random_state = 42)


regressor_red.fit(X_train, y_train)
y_pred = pd.Series(regressor_red.predict(X_test))
#y_pred.loc[X_test[X_test['TP_STATUS_REDACAO']!=1].reset_index(drop=True).index] = 0
np.sqrt(mean_squared_error(y_test, y_pred))



113.53623400923703

## Predição das Notas de Redação

In [ ]:
submissions = pd.read_csv('./submissions.csv', engine='c')
df_rd = reduce_mem_usage( pd.read_csv('../input/test.csv', engine='c', usecols=list_vars) )

In [ ]:
## --- substitui valores NaN (variáveis numéricas)  por inteiro arbitrário 
for coluna in df_rd.loc[:2,~df_rd.columns.isin(list_toenc)].columns:
    df_rd[coluna] = df_rd[coluna].fillna(-32768).astype('int16')
    
i=0
for coluna in list_toenc:
    df_rd[coluna] = df_rd[coluna].astype(str).fillna("missing").replace("nan", "missing").astype('category')
    df_rd[coluna] = encoders[i].transform(df_rd[coluna])
    i+=1

In [ ]:
## --- redução de cardinalidade

df_rd['Q006'] = df_rd['Q006'].apply(lambda x: 1 if x<=8 else 9)
df_rd['Q024'] = df_rd['Q024'].apply(lambda x: x if x<3 else 3)
df_rd['Q004'] = df_rd['Q004'].apply(lambda x: 0 if x==0 else
                                        1 if (x==1) | (x==2) | (x==5) else x-1)
df_rd['Q002'] = df_rd['Q002'].apply(lambda x: 1 if (x==1) | (x==7) else x)

df_rd['Q003'] = df_rd['Q003'].apply(lambda x: 0 if x==0 else
                                        1 if (x==1) | (x==2) | (x==5) else x-1)

In [ ]:
## clusterização
df_rd['CLUSTER'] = model.predict(df_rd[ft_clust])
## predição
submissions['NU_NOTA_REDACAO'] = regressor_red.predict(df_rd[np.append(ft_rd,'CLUSTER')])
submissions['NU_NOTA_REDACAO'].iloc[df_rd[df_rd['TP_STATUS_REDACAO']!=1].index] = 0
submissions.to_csv('./submissions.csv', index=False)